In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import power_transform
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.base import clone
from sklearn.impute import SimpleImputer

import lightgbm as lgbm
import optuna  # pip install optuna

In [2]:
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def objective(base_model, trial, X, y):
    
    
    if type(base_model) == Pipeline:
        param_grid = {
            "lgbmclassifier__device_type": trial.suggest_categorical("device_type", ['gpu']),
            "lgbmclassifier__n_estimators": trial.suggest_int("n_estimators", 10, 10000),
            "lgbmclassifier__learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "lgbmclassifier__num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "lgbmclassifier__max_depth": trial.suggest_int("max_depth", 3, 12),
            "lgbmclassifier__min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lgbmclassifier__lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lgbmclassifier__lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "lgbmclassifier__min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "lgbmclassifier__bagging_fraction": trial.suggest_float(
                "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "lgbmclassifier__bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
            "lgbmclassifier__feature_fraction": trial.suggest_float(
                "feature_fraction", 0.2, 0.95, step=0.1
            ),
        }
    else:
        param_grid = {
            "device_type": trial.suggest_categorical("device_type", ['gpu']),
            "n_estimators": trial.suggest_int("n_estimators", 10, 10000),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float(
                "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
            "feature_fraction": trial.suggest_float(
                "feature_fraction", 0.2, 0.95, step=0.1
            ),
        }
    
    #cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    #cv_scores = np.empty(5)
    #for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    #model = lgbm.LGBMClassifier(objective="binary", **param_grid)
    
    model = clone(base_model)
    model.set_params(**param_grid)
    
    if type(base_model) == Pipeline:
        model.fit(
            X_train,
            y_train,
            lgbmclassifier__eval_set=[(X_test, y_test)],
            lgbmclassifier__eval_metric="auc",
            lgbmclassifier__early_stopping_rounds=100,
            lgbmclassifier__callbacks=[
                LightGBMPruningCallback(trial, "auc")
            ],  # Add a pruning callback
            lgbmclassifier__verbose = 0
        )
    else: 
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "auc")
            ],  # Add a pruning callback
            verbose = 0
        )
    preds = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, preds)

In [3]:
def tune_and_train_lightgbm_model(base_model, X, y, study_name = "LGBM Classifier"):
    
    study = optuna.create_study(direction="maximize", study_name=study_name)
    func = lambda trial: objective(base_model, trial, X, y)
    study.optimize(func, n_trials=100)
    
    print(f"\tBest value (AUC): {study.best_value:.5f}")
    print(f"\tBest params:")

    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")
        
    model = clone(base_model)
    model.set_params(**study.best_params)
    model.fit(X, y)
    
    return model, study

# Get Data

In [4]:
train = pd.read_csv("Data/train.csv", index_col = 0)
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [5]:
train.shape

(957919, 119)

In [6]:
test = pd.read_csv("Data/test.csv", index_col = 0)
test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
id,,,,,,,,,,,,,,,,,,,,,
957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.000,...,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.430,...,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.810,...,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797
957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,-26.473,...,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910
957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,5999.400,...,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796


In [ ]:
sample_submission = pd.read_csv("Data/sample_solution.csv")

In [7]:
def feature_generation(data):
    
    data = data.copy()

    # add feature with number of na's per row + standard deviation
    features = [col for col in data.columns if 'f' in col] # if 'f' is in the name
    data['count_na'] = data[features].isna().sum(axis=1)

    features += ['count_na']

    # now do imputation with mean
    data[features] = data[features].fillna(data[features].mean())

    return data

train_w_nan = feature_generation(train)

In [8]:
train_w_nan.to_csv("Data/train_w_nan.csv")

In [13]:
test_w_nan = feature_generation(test)
test_w_nan.to_csv("Data/test_w_nan.csv")

# Fill in missing values with mean

In [9]:
X = train_w_nan.drop("claim", axis = 1)
y = train_w_nan.claim

In [10]:
model = make_pipeline(SimpleImputer(strategy = 'mean'), lgbm.LGBMClassifier(objective="binary"))

trained_model, study = tune_and_train_lightgbm_model(model, X, y, study_name = "LGBM Classifier with mean imputation")

[I 2021-09-11 22:42:56,609] A new study created in memory with name: LGBM Classifier with mean imputation
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=7100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7100
[LightGBM] [Warning] min_gain_to_split is set=3.7415490955668744, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.7415490955668744
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 22:43:25,159] Trial 0 finished with value: 0.8093150083264089 and parameters: {'device_type': 'gpu', 'n_estimators': 6681, 'learning_rate': 0.28441310612188986, 'num_leaves': 1180, 'max_depth': 9, 'min_data_in_leaf': 7100, 'lambda_l1': 5, 'lambda_l2': 15, 'min_gain_to_split': 3.7415490955668744, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 0 with value: 0.8093150083264089.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=9900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9900
[LightGBM] [Warning] min_gain_to_split is set=4.012947696524615, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.012947696524615
[LightGBM] [Warning] lambda_l1 is set=65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=65
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=40, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 22:55:04,856] Trial 1 finished with value: 0.8149850101625152 and parameters: {'device_type': 'gpu', 'n_estimators': 6127, 'learning_rate': 0.013771215541806329, 'num_leaves': 2640, 'max_depth': 8, 'min_data_in_leaf': 9900, 'lambda_l1': 65, 'lambda_l2': 40, 'min_gain_to_split': 4.012947696524615, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.8149850101625152.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[I 2021-09-11 23:13:09,905] Trial 39 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:13:19,732] Trial 40 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:13:28,092] Trial 41 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divis

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=7500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7500
[LightGBM] [Warning] min_gain_to_split is set=6.520587022281271, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.520587022281271
[LightGBM] [Warning] lambda_l1 is set=30, reg_alpha=0.0 will be ignored. Current value: lambda_l1=30
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 23:21:35,994] Trial 88 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:21:43,739] Trial 89 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:21:50,760] Trial 90 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divis

	Best value (AUC): 0.81573
	Best params:
		device_type: gpu
		n_estimators: 8017
		learning_rate: 0.2484019052000147
		num_leaves: 1300
		max_depth: 10
		min_data_in_leaf: 8700
		lambda_l1: 25
		lambda_l2: 20
		min_gain_to_split: 6.644633325828346
		bagging_fraction: 0.8
		bagging_freq: 1
		feature_fraction: 0.30000000000000004


ValueError: Invalid parameter device_type for estimator Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('lgbmclassifier', LGBMClassifier(objective='binary'))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [12]:
best_params = {
            "lgbmclassifier__device_type": 'gpu',
            "lgbmclassifier__n_estimators": 8017,
            "lgbmclassifier__learning_rate": 0.2484019052000147,
            "lgbmclassifier__num_leaves": 1300,
            "lgbmclassifier__max_depth": 10,
            "lgbmclassifier__min_data_in_leaf": 8700,
            "lgbmclassifier__lambda_l1": 25,
            "lgbmclassifier__lambda_l2": 20,
            "lgbmclassifier__min_gain_to_split": 6.644633325828346,
            "lgbmclassifier__bagging_fraction": 0.8,
            "lgbmclassifier__bagging_freq": 1,
            "lgbmclassifier__feature_fraction": 0.3
        }

model.set_params(**best_params)
model.fit(X, y)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('lgbmclassifier',
                 LGBMClassifier(bagging_fraction=0.8, bagging_freq=1,
                                device_type='gpu', feature_fraction=0.3,
                                lambda_l1=25, lambda_l2=20,
                                learning_rate=0.2484019052000147, max_depth=10,
                                min_data_in_leaf=8700,
                                min_gain_to_split=6.644633325828346,
                                n_estimators=8017, num_leaves=1300,
                                objective='binary'))])

In [17]:
predictions = model.predict_proba(test_w_nan)

In [19]:
sample_submission.claim = predictions[:, 1]
sample_submission

,id,claim
0,957919,0.603748
1,957920,0.126944
2,957921,0.649066
3,957922,0.100918
4,957923,0.130741
...,...,...
493469,1451388,0.824863
493470,1451389,0.116021
493471,1451390,0.694402
493472,1451391,0.132100


In [20]:
sample_submission.to_csv("Submissions/lightgbm_w_mean_imputation.csv", index = False)

# Drop missing values

In [21]:
temp = train_w_nan.dropna().copy()
X = temp.drop("claim", axis = 1)
y = temp.claim

In [23]:
base_model = lgbm.LGBMClassifier(objective="binary")
study = optuna.create_study(direction="maximize", study_name="LightGBM dropping missing values")
func = lambda trial: objective(base_model, trial, X, y)
study.optimize(func, n_trials=100)

print(f"\tBest value (AUC): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

model = clone(base_model)
model.set_params(**study.best_params)
model.fit(X, y)

[I 2021-09-11 23:44:17,876] A new study created in memory with name: LightGBM dropping missing values
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=3400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3400
[LightGBM] [Warning] min_gain_to_split is set=2.6564271439803417, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.6564271439803417
[LightGBM] [Warning] lambda_l1 is set=90, reg_alpha=0.0 will be ignored. Current value: lambda_l1=90
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=55, reg_lambda=0.0 will be ignored. Current value: lambda_l2=55
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 23:47:08,969] Trial 0 finished with value: 0.815019711281604 and parameters: {'device_type': 'gpu', 'n_estimators': 9218, 'learning_rate': 0.22573906357492232, 'num_leaves': 2000, 'max_depth': 7, 'min_data_in_leaf': 3400, 'lambda_l1': 90, 'lambda_l2': 55, 'min_gain_to_split': 2.6564271439803417, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 0.815019711281604.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=9100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9100
[LightGBM] [Warning] min_gain_to_split is set=11.032217259948618, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.032217259948618
[LightGBM] [Warning] lambda_l1 is set=75, reg_alpha=0.0 will be ignored. Current value: lambda_l1=75
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 23:48:03,621] Trial 1 finished with value: 0.8134557544000776 and parameters: {'device_type': 'gpu', 'n_estimators': 773, 'learning_rate': 0.2757912114441294, 'num_leaves': 1240, 'max_depth': 3, 'min_data_in_leaf': 9100, 'lambda_l1': 75, 'lambda_l2': 5, 'min_gain_to_split': 11.032217259948618, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 0.815019711281604.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[I 2021-09-11 23:53:03,973] Trial 4 finished with value: 0.8133397251727377 and parameters: {'device_type': 'gpu', 'n_estimators': 2224, 'learning_rate': 0.2813898759545546, 'num_leaves': 2300, 'max_depth': 7, 'min_data_in_leaf': 3800, 'lambda_l1': 95, 'lambda_l2': 60, 'min_gain_to_split': 7.36129419584745, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 2 with value: 0.8153911644695946.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=5900, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5900
[LightGBM] [Warning] min_gain_to_split is set=9.217117903777828, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9.217117903777828
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] lambda_l2 is set=20, reg_lambda=0.0 will be ignored. Current value: lambda_l2=20
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-11 23:53:10,825] Trial 5 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:53:18,237] Trial 6 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-11 23:53:25,061] Trial 7 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisibl

	Best value (AUC): 0.81539
	Best params:
		device_type: gpu
		n_estimators: 3052
		learning_rate: 0.10349950719079135
		num_leaves: 540
		max_depth: 4
		min_data_in_leaf: 5700
		lambda_l1: 55
		lambda_l2: 85
		min_gain_to_split: 2.4288895631404035
		bagging_fraction: 0.4
		bagging_freq: 1
		feature_fraction: 0.4


LGBMClassifier(bagging_fraction=0.4, bagging_freq=1, device_type='gpu',
               feature_fraction=0.4, lambda_l1=55, lambda_l2=85,
               learning_rate=0.10349950719079135, max_depth=4,
               min_data_in_leaf=5700, min_gain_to_split=2.4288895631404035,
               n_estimators=3052, num_leaves=540, objective='binary')

In [24]:
predictions = model.predict_proba(test_w_nan)

In [25]:
sample_submission.claim = predictions[:, 1]
sample_submission

,id,claim
0,957919,0.566248
1,957920,0.121081
2,957921,0.660913
3,957922,0.137200
4,957923,0.157628
...,...,...
493469,1451388,0.832642
493470,1451389,0.103087
493471,1451390,0.794027
493472,1451391,0.142980


In [26]:
sample_submission.to_csv("Submissions/lightgbm_w_dropping_missing.csv", index = False)

# Generate new features as required

In [72]:
def split_multimodal_features(data):
    
    df = data.copy()
    
    def split_feature(df, feature, threshold):

        df.loc[df[feature] <= threshold, feature + '_0'] = df[feature]
        df.loc[df[feature] > threshold, feature + '_1'] = df[feature]
        df = df.drop(feature, axis = 1)

        return df
    
    df = split_feature(df, 'f2', 0.025)
    df = split_feature(df, 'f5', 0.05)
    df = split_feature(df, 'f11', 0)
    df = split_feature(df, 'f13', 0.04)
    df = split_feature(df, 'f23', 6.2)
    df = split_feature(df, 'f26', 1e13)
    df = split_feature(df, 'f29', 0.05)
    df = split_feature(df, 'f29_1', 0.5)
    df = split_feature(df, 'f40', 0.05)
    df = split_feature(df, 'f40_1', 0.5)
    df = split_feature(df, 'f42', 0.3)
    df = split_feature(df, 'f42_1', 0.7)
    df = split_feature(df, 'f47', 0)
    df = split_feature(df, 'f49', -0.05)
    df = split_feature(df, 'f49_1', 0.05)
    df = split_feature(df, 'f50', 0.05)
    df = split_feature(df, 'f55', 0.035)
    df = split_feature(df, 'f58', -0.97)
    df = split_feature(df, 'f65', 40000)
    df = split_feature(df, 'f66', 4)
    df = split_feature(df, 'f70', 0.5)
    df = split_feature(df, 'f75', 0.5)
    df = split_feature(df, 'f75_0', 0.05)
    df = split_feature(df, 'f75_1', 0.95)
    df = split_feature(df, 'f91', 0.1)
    df = split_feature(df, 'f112', 30)
    
    return df

In [73]:
train_w_nan_new_features = split_multimodal_features(train_w_nan)
test_w_nan_new_features = split_multimodal_features(test_w_nan)

In [75]:
X = train_w_nan_new_features.drop('claim', axis = 1)
y = train_w_nan_new_features.claim

trained_model, study = tune_and_train_lightgbm_model(model, X, y, study_name = "LGBM Classifier with new features")

[I 2021-09-12 22:26:17,852] A new study created in memory with name: LGBM Classifier with new features
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=6300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6300
[LightGBM] [Warning] min_gain_to_split is set=2.1303387759183003, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.1303387759183003
[LightGBM] [Warning] lambda_l1 is set=50, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=90, reg_lambda=0.0 will be ignored. Current value: lambda_l2=90
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 22:27:48,859] Trial 0 finished with value: 0.8144281010472123 and parameters: {'device_type': 'gpu', 'n_estimators': 2668, 'learning_rate': 0.28861785824133335, 'num_leaves': 1020, 'max_depth': 7, 'min_data_in_leaf': 6300, 'lambda_l1': 50, 'lambda_l2': 90, 'min_gain_to_split': 2.1303387759183003, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.8144281010472123.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_gain_to_split is set=4.043964359011708, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.043964359011708
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 22:28:45,091] Trial 1 finished with value: 0.8112886733473452 and parameters: {'device_type': 'gpu', 'n_estimators': 9982, 'learning_rate': 0.24661482314007654, 'num_leaves': 1400, 'max_depth': 8, 'min_data_in_leaf': 500, 'lambda_l1': 10, 'lambda_l2': 0, 'min_gain_to_split': 4.043964359011708, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 0 with value: 0.8144281010472123.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=4400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4400
[LightGBM] [Warning] min_gain_to_split is set=9.247562406131543, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9.247562406131543
[LightGBM] [Warning] lambda_l1 is set=10, reg_alpha=0.0 will be ignored. Current value: lambda_l1=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=20, reg_lambda=0.0 will be ignored. Current value: lambda_l2=20
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:19:19,647] Trial 89 pruned. Trial was pruned at iteration 4.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:19:26,523] Trial 90 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:20:50,406] Trial 91 finished with value: 0.8150337310627702 and parameters: {'device_type': 'gpu', 'n_estimators': 1549, 'learning_rate': 0.28488128568706805, 'num_leaves': 2260, 'max_depth': 8, 'min_data_in_leaf': 4200, 'lambda_l1': 5, 'lambda_l2': 10, 'min_gain_to_split'

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=4200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4200
[LightGBM] [Warning] min_gain_to_split is set=4.201529677886941, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.201529677886941
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:21:10,804] Trial 92 pruned. Trial was pruned at iteration 44.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:21:17,611] Trial 93 pruned. Trial was pruned at iteration 1.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:22:04,800] Trial 94 pruned. Trial was pruned at iteration 187.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not di

	Best value (AUC): 0.81542
	Best params:
		device_type: gpu
		n_estimators: 1625
		learning_rate: 0.26486470340395635
		num_leaves: 2820
		max_depth: 8
		min_data_in_leaf: 5300
		lambda_l1: 15
		lambda_l2: 0
		min_gain_to_split: 6.398418703243587
		bagging_fraction: 0.9
		bagging_freq: 1
		feature_fraction: 0.8


In [81]:
func = lambda trial: objective(base_model, trial, X, y)
study.optimize(func, n_trials=100)

C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=3500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3500
[LightGBM] [Warning] min_gain_to_split is set=4.517784753049794, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.517784753049794
[LightGBM] [Warning] lambda_l1 is set=25, reg_alpha=0.0 will be ignored. Current value: lambda_l1=25
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:31:52,459] Trial 100 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:32:01,213] Trial 101 pruned. Trial was pruned at iteration 5.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:32:10,798] Trial 102 pruned. Trial was pruned at iteration 8.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not di

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=3700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3700
[LightGBM] [Warning] min_gain_to_split is set=5.572034804184396, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.572034804184396
[LightGBM] [Warning] lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:33:49,468] Trial 105 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:33:56,411] Trial 106 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:34:04,411] Trial 107 pruned. Trial was pruned at iteration 1.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not di

...

[I 2021-09-12 23:43:08,433] Trial 143 pruned. Trial was pruned at iteration 8.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:43:17,955] Trial 144 pruned. Trial was pruned at iteration 1.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:43:26,883] Trial 145 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not di

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=3100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3100
[LightGBM] [Warning] min_gain_to_split is set=5.209620605024289, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.209620605024289
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:46:24,939] Trial 150 pruned. Trial was pruned at iteration 16.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:46:36,263] Trial 151 pruned. Trial was pruned at iteration 5.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:46:45,506] Trial 152 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not d

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=3500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3500
[LightGBM] [Warning] min_gain_to_split is set=3.3096240886316854, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.3096240886316854
[LightGBM] [Warning] lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2021-09-12 23:55:13,204] Trial 188 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:55:22,914] Trial 189 pruned. Trial was pruned at iteration 2.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2021-09-12 23:55:32,001] Trial 190 pruned. Trial was pruned at iteration 0.
C:\Users\I539756\.virtualenvs\202109_TabularPlaygroundSeries-75bSHUOP\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not di

In [82]:
    print(f"\tBest value (AUC): {study.best_value:.5f}")
    print(f"\tBest params:")

    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")
        
    model = clone(base_model)
    model.set_params(**study.best_params)
    model.fit(X, y)

	Best value (AUC): 0.81566
	Best params:
		device_type: gpu
		n_estimators: 431
		learning_rate: 0.27510378410487624
		num_leaves: 2140
		max_depth: 9
		min_data_in_leaf: 6000
		lambda_l1: 5
		lambda_l2: 15
		min_gain_to_split: 6.583518223586768
		bagging_fraction: 0.9
		bagging_freq: 1
		feature_fraction: 0.8


LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, device_type='gpu',
               feature_fraction=0.8, lambda_l1=5, lambda_l2=15,
               learning_rate=0.27510378410487624, max_depth=9,
               min_data_in_leaf=6000, min_gain_to_split=6.583518223586768,
               n_estimators=431, num_leaves=2140, objective='binary')

In [ ]:
predictions = trained_model.predict_proba(test_w_nan_new_features)

In [83]:
sample_submission.claim = predictions[:, 1]
sample_submission

,id,claim
0,957919,0.560970
1,957920,0.121897
2,957921,0.602786
3,957922,0.130965
4,957923,0.131961
...,...,...
493469,1451388,0.850729
493470,1451389,0.115525
493471,1451390,0.754884
493472,1451391,0.138521


In [84]:
sample_submission.to_csv("Submissions/lightgbm_new_features_extra_tuning.csv", index = False)